In [30]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

In [31]:
# Read the files into DataFrames
df_per = pd.read_csv("Deliv_PER.csv")
df_hh = pd.read_csv("Deliv_HH.csv")



In [32]:
print(df_hh.columns.tolist())
print(df_per.columns.tolist())

['SAMPN', 'RECMODE', 'RETMODE', 'INCEN', 'ILANG', 'CTFIP', 'AREA', 'STRATA', 'STYPE', 'CEC', 'GTYPE', 'GFLAG', 'RIBUS', 'HHVEH', 'HHBIC', 'VEHNEW', 'BUYER1', 'BUYER2', 'BUYER3', 'BUYER4', 'BUYER5', 'BUYER6', 'BUYER7', 'BUYER8', 'RESTY', 'O_RESTY', 'OWN', 'O_OWN', 'TEN', 'PREVCITY', 'PREVSTAT', 'PREVZIP', 'PHLNS', 'INCOM', 'HHSIZ', 'NONRELAT', 'HHEMP', 'HHSTU', 'HHLIC', 'RECDate', 'ASSN', 'DOW', 'HTRIPS', 'HCITY', 'HSTAT', 'HZIP', 'HHNOV1', 'HHNOV2', 'HHNOV3', 'HHNOV4', 'HHNOV5', 'HHNOV6', 'HHNOV7', 'HHNOV8', 'VEHOP', 'VEHPR', 'VEDTE', 'CMPLD', 'LDPER1', 'LDPER2', 'LDPER3', 'LDPER4', 'LDPER5', 'LDPER6', 'LDPER7', 'LDPER8', 'LDTRP', 'LDFlag', 'HPFlag', 'HH_Complete', 'GPS_Complete', 'HCTRACT', 'HPrimaryCity', 'HSTFIP', 'MTC_FInalFlag', 'HHWGT', 'EXPHHWGT', 'GPS_MTC']
['ID', 'SAMPN', 'PERNO', 'RELAT', 'GEND', 'AGE', 'AGEB', 'HISP', 'RACE1', 'RACE2', 'RACE3', 'RACE4', 'O_RACE', 'NTVTY', 'CNTRY', 'LIC', 'USER', 'TRANS', 'TPTYP1', 'TPTYP2', 'TPTYP3', 'TPTYP4', 'TPTYP5', 'TPTYP6', 'TPTYP7', '

In [33]:
def drop_columns_with_missing_values(full_df, missing_value_threshold=80):
    """
    Drops columns from a DataFrame that have more than a specified percentage of missing values.
    
    Parameters:
    - full_df (pd.DataFrame): The input DataFrame.
    - missing_value_threshold (float): The percentage threshold for missing values above which columns will be dropped (default is 99.9%).
    
    Returns:
    - pd.DataFrame: A DataFrame with the columns with excessive missing values removed.
    """
    # Calculate the percentage of missing values in each column
    missing_percentage = full_df.isnull().mean() * 100
    
    # Identify columns with more missing values than the threshold
    columns_to_drop = missing_percentage[missing_percentage > missing_value_threshold].index.tolist()
    print(columns_to_drop)
    # Drop the identified columns and return the modified DataFrame
    return full_df.drop(columns=columns_to_drop)

In [34]:
import re

def categorize_and_merge_by_prefix_with_number(df):
    # Create a dictionary to store merged columns
    merged_columns = {}

    # Initialize a dictionary to group columns by the prefix (first 4 characters)
    column_groups = {}

    # Loop through the column names and group them based on their prefix and numeric suffix
    for col in df.columns:
        try:
            # Check if the column name starts with 4 or more characters and ends with a number
            match = re.match(r'^([A-Za-z]{4,})(\d+)$', col)
            if match:
                prefix = match.group(1)  # The prefix (first 4 or more characters)
                if prefix not in column_groups:
                    column_groups[prefix] = []
                column_groups[prefix].append(col)
        except Exception as e:
            print(f"Error processing column {col}: {e}")
    
    # Merge the grouped columns into new columns as lists
    for prefix, columns in column_groups.items():
        if len(columns) > 1:  # Only merge if there is more than one column with the same prefix
            merged_columns[f'Merged_{prefix}'] = df[columns].apply(lambda row: row.tolist(), axis=1)
            # Drop the original columns after merging
            df.drop(columns=columns, inplace=True)
    # Add the merged columns to the DataFrame
    df = df.assign(**merged_columns)

    return df, column_groups


In [35]:
def drop_columns_starting_with_LD(df):
    """
    Drops columns in the dataframe whose names start with 'LD'.
    
    Parameters:
    - df (pd.DataFrame): The dataframe from which to drop columns.
    
    Returns:
    - pd.DataFrame: The updated dataframe with the specified columns dropped.
    """
    # Identify columns that start with 'LD'
    columns_to_drop = [col for col in df.columns if col.startswith('LD')]
    
    # Drop the identified columns
    df = df.drop(columns=columns_to_drop)
    
    return df

In [ ]:
def drop_specific_columns(df):
    """
    Drops specific columns from the DataFrame if they exist. These are out of scope 

    Parameters:
    - df (pd.DataFrame): The DataFrame from which columns will be dropped.
    - columns_to_drop (list): List of column names to drop.

    Returns:
    - pd.DataFrame: The DataFrame with specified columns dropped.
    """
    columns_to_drop = [
    'VEHNO', 'O_PRKTY', 'O_TRANSYS', 'TripDistanceFlag', 'AirTripDistance', 'CTFIP', 'TRACT', 'PSTFIP', 'PERWGT', 
    'EXPPERWGT', 'RECMODE', 'RETMODE', 'STYPE', 'RIBUS', 'TEN', 'PREVCITY', 'PREVSTAT', 'PREVZIP', 'PHLNS', 'VEDTE', 
    'CMPLD', 'HH_Complete', 'HSTFIP', 'MTC_FInalFlag', 'ID', 'TPTYP1', 'TPTYP2', 'TPTYP3', 'TPTYP4', 'TPTYP5', 'TPTYP6', 
    'TPTYP7', 'O_TPTYP', 'CLIP1', 'CLIP2', 'CLIP3', 'COMP', 'MET', 'PASSTL', 'WNAME', 'WCITY', 'WSTAT', 'WXCORD', 
    'WYCORD', 'DTYPE1', 'DTYPE2', 'DTYPE3', 'DTYPE4', 'DTYPE5', 'DTYPE6', 'DTYPE7', 'INTRV', 'CMPLG', 'HVLOG', 'WCTFIP', 
    'WTRACT', 'SCTFIP', 'STRACT', 'WPrimaryCity', 'WSTFIP', 'W2PrimaryCity', 'W2STFIP', 'SPrimaryCity', 'SSTFIP', 'BODY', 
    'O_BODY', 'FUELT1', 'FUELT2', 'FUELT3', 'FUELT4', 'FUELT5', 'FUELT6', 'O_FUELT', 'CIGLT', 'VEHAQ', 'VEHOWN', 'O_VEHOWN', 
    'VEHINS', 'VEHOBD', 'VEHTRN', 'VEHDRT', 'O_VEHDRT', 'VEHCYL', 'O_VEHCYL', 'VEHOUT', 'VEHVLT', 'WYCNTV', 'O_WYCNTV', 
    'PRESCH', 'O_PRESCH', 'SNAME'
    ]
    # Drop columns that are in the list and exist in the DataFrame
    columns_to_drop_in_df = [col for col in columns_to_drop if col in df.columns]
    print(columns_to_drop_in_df)
    # Drop the identified columns
    df_dropped = df.drop(columns=columns_to_drop_in_df)
    
    return df_dropped

# Example usage:


In [43]:

df_hh, column_groups = categorize_and_merge_by_prefix_with_number(df_hh)
df_hh = drop_columns_starting_with_LD(df_hh)
df_hh = drop_columns_with_missing_values(df_hh)
df_hh = drop_specific_columns(df_hh)

[]
['CTFIP', 'RECMODE', 'RETMODE', 'STYPE', 'RIBUS', 'TEN', 'PREVCITY', 'PREVSTAT', 'PREVZIP', 'PHLNS', 'CMPLD', 'HH_Complete', 'HSTFIP', 'MTC_FInalFlag']


In [44]:
#merge first 
#then drop and lastly cremove missins 
df_per, column_groups = categorize_and_merge_by_prefix_with_number(df_per)
df_per = drop_columns_starting_with_LD(df_per)
df_per = drop_columns_with_missing_values(df_per)
df_per = drop_specific_columns(df_per)

[]
['PERWGT', 'EXPPERWGT', 'ID', 'PASSTL', 'WNAME', 'WCITY', 'WSTAT', 'WXCORD', 'WYCORD', 'INTRV', 'CMPLG', 'HVLOG', 'WCTFIP', 'WTRACT', 'SCTFIP', 'STRACT', 'WPrimaryCity', 'WSTFIP', 'SPrimaryCity', 'SSTFIP', 'SNAME']


In [39]:
print(df_hh.columns.tolist())
print(df_per.columns.tolist())

['SAMPN', 'RECMODE', 'RETMODE', 'INCEN', 'ILANG', 'CTFIP', 'AREA', 'STRATA', 'STYPE', 'GTYPE', 'RIBUS', 'HHVEH', 'HHBIC', 'VEHNEW', 'RESTY', 'OWN', 'TEN', 'PREVCITY', 'PREVSTAT', 'PREVZIP', 'PHLNS', 'INCOM', 'HHSIZ', 'HHEMP', 'HHSTU', 'HHLIC', 'RECDate', 'ASSN', 'DOW', 'HTRIPS', 'HCITY', 'HSTAT', 'HZIP', 'VEHOP', 'CMPLD', 'HPFlag', 'HH_Complete', 'HCTRACT', 'HPrimaryCity', 'HSTFIP', 'MTC_FInalFlag', 'HHWGT', 'EXPHHWGT', 'Merged_BUYER', 'Merged_HHNOV', 'Merged_LDPER']
['ID', 'SAMPN', 'PERNO', 'RELAT', 'GEND', 'AGE', 'HISP', 'NTVTY', 'LIC', 'USER', 'TRANS', 'PASSTL', 'FLEX', 'EMPLY', 'WKSTAT', 'JOBS', 'WLOC', 'WNAME', 'WCITY', 'WSTAT', 'WZIP', 'WXCORD', 'WYCORD', 'WDAYS', 'HOURS', 'WSCHED', 'COMPR', 'WMODE', 'INDUS', 'OCCUP', 'DISAB', 'TTRIP', 'TRNSUB', 'WTRIP', 'BTRIP', 'STUDE', 'SCHOL', 'SNAME', 'SCITY', 'SSTAT', 'SZIP', 'SXCORD', 'SYCORD', 'SMODE', 'EDUCA', 'INTRV', 'CMPLG', 'HVLOG', 'PTRIPS', 'TOLLF', 'HOVL', 'NOGOWHY', 'Moto_trip', 'WCTFIP', 'WTRACT', 'SCTFIP', 'STRACT', 'WPrimaryCi

In [ ]:
# Merge the DataFrames on the "SAMPN" column
merged_df = pd.merge(df_per, df_hh, on="SAMPN", how="left")

# Display the first few rows of the merged DataFrame
print(merged_df.head())



       SAMPN  PERNO  RELAT  GEND   AGE  HISP  NTVTY  LIC  USER  TRANS  ...  \
0  1046924.0    1.0    1.0   2.0  77.0   2.0    1.0  1.0   1.0    2.0  ...   
1  1046924.0    2.0    2.0   1.0  77.0   2.0    1.0  1.0   1.0    2.0  ...   
2  1047092.0    1.0    1.0   2.0  50.0   2.0    1.0  1.0   1.0    2.0  ...   
3  1047092.0    2.0    2.0   1.0  51.0   2.0    1.0  1.0   2.0    2.0  ...   
4  1048704.0    1.0    1.0   2.0  36.0   1.0    2.0  2.0   NaN    2.0  ...   

    HZIP  VEHOP  HPFlag  HCTRACT  HPrimaryCity     HHWGT    EXPHHWGT  \
0  92061    2.0       2    19101  PAUMA VALLEY  2.134283  625.389658   
1  92061    2.0       2    19101  PAUMA VALLEY  2.134283  625.389658   
2  94501    4.0       2   427700       ALAMEDA  0.784909  229.994688   
3  94501    4.0       2   427700       ALAMEDA  0.784909  229.994688   
4  91763    NaN       1      207     MONTCLAIR  1.853113  543.001077   

                               Merged_BUYER  \
0  [nan, nan, nan, nan, nan, nan, nan, nan]   
1  [

In [ ]:
# Save the merged DataFrame to a new file
merged_df.to_csv("Merged_Deliv.csv", index=False)